In [90]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [91]:
df = pd.read_csv('~/Dropbox/Element/Phase2/TradesJan22.csv')
df = df[df['fillQuantity'] > 0]

In [92]:
parents = df['baseParentNumber'].unique()
df = df[df['baseParentNumber'] == parents[1]]

In [93]:
# Deal with apparent float precision issues to return true penny prices
round_cols = [col for col in df.columns[3:] if df[col].dtype=='float64']
round_cols = [col for col in round_cols if ('Vol' not in col) and ('Prob' not in col) and ('Mark' not in col)]
for col in round_cols:
    df[col] = df[col].apply(lambda x: round(x, 2))
    
# Convert Date Cols and add in microsecond cols
def col_to_time(col):
    df[col] = df[col].apply(pd.to_datetime)
    df[col] = df[col].dt.tz_localize('America/Chicago').dt.tz_convert('America/New_York')
    s = df[col+'_us'].apply(pd.Timedelta, unit='micros')
    df[col] = df[col] + s
    
for col in ['childDttm', 'fillTransactDttm']:
    col_to_time(col)

df['parentDttm'] = df['parentDttm'].apply(lambda s: pd.to_datetime(s))
df['parentDttm'] = df['parentDttm'].dt.tz_localize('America/Chicago').dt.tz_convert('America/New_York')

In [94]:
start = df.loc[df.index[0], 'parentDttm']
arrival_bid = df.loc[df.index[0], 'parentBid']
arrival_ask = df.loc[df.index[0], 'parentAsk']
arrival_mid = (arrival_bid + arrival_ask) / 2
arrival_mark = df.loc[df.index[0], 'parentMark']
arrival_ul_mid = (df.loc[df.index[0], 'parentUBid'] + df.loc[df.index[0], 'parentUAsk']) / 2
delta = df.loc[df.index[0], 'fillDe']
vega = df.loc[df.index[0], 'fillVe']

In [95]:
# Construct a delta-adjusted version of the data, referenced to arrival_ul_mid
cols = ['fillTransactDttm', 'orderSide', 'fillBid', 'fillAsk', 
        'fillMark', 'fillPrice', 'fillLimitRefUPrc', 'fillQuantity']
df_adj = pd.DataFrame(index = df.index, columns=cols)

for col in cols[2:-2]:
    df_adj[col] = df[col] - delta * ((df['fillUBid'] + df['fillUAsk'])/2 - arrival_ul_mid)
    
for col in cols[:2] + cols[-2:]:
    df_adj[col] = df[col]

In [96]:
# Convert the delta-adjusted prices to vols
first_trade_vol = df.loc[df.index[0], 'fillVol']
first_trade_adj_px = df_adj.loc[df_adj.index[0], 'fillPrice']

arrival_mid_vol = first_trade_vol + (arrival_mid - first_trade_adj_px) / (100 * vega)
arrival_mark_vol = first_trade_vol + (arrival_mark - first_trade_adj_px) / (100 * vega)

df_vol = pd.DataFrame(index = df.index, columns=cols)
for col in cols[2:-2]:
    df_vol[col] = arrival_mid_vol + (df_adj[col] - arrival_mid) / (100 * vega)

for col in cols[:2] + cols[-2:]:
    df_vol[col] = df[col]

df_vol[cols[-1]] = df[cols[-1]]

In [97]:
df = df[cols]
df['cumQuantity'] = df['fillQuantity'].cumsum()
df_adj['cumQuantity'] = df['cumQuantity']
df_vol['cumQuantity'] = df['cumQuantity']

In [102]:
def plot_fill_graph(df, pct_y=False):
    max_fill = df['fillQuantity'].max()
    scale_arrows = False

    fig = make_subplots(rows=2, cols=1, row_heights = [1, 0.3], vertical_spacing=0.02,
                        shared_xaxes=True, specs=[[{'secondary_y': True}],
                                              [{'secondary_y': True}]])
    fig.add_trace(go.Scatter(x=df['fillTransactDttm'], y=df['fillBid'], name='Bid',
                                 line = dict(color='royalblue', width=1, dash='solid')), row=1, col=1)
    fig.add_trace(go.Scatter(x=df['fillTransactDttm'], y=df['fillAsk'], name='Ask',
                                 line = dict(color='green', width=1, dash='solid')), row=1, col=1)
    fig.add_trace(go.Scatter(x=df['fillTransactDttm'], y=df['fillMark'], name='SR Mark',
                                 line = dict(color='magenta', width=1, dash='dot')), row=1, col=1)
    fig.add_trace(go.Scatter(x=df['fillTransactDttm'], y=df['cumQuantity'], name='Cumulative Fills',
                                 line = dict(color='black', width=1, dash='dot')), secondary_y=True, row=1, col=1)
    fig.add_trace(go.Scatter(x=df['fillTransactDttm'], y=df['fillAsk']-df['fillBid'], name='Spread',
                                 line = dict(color='royalblue', width=1, dash='solid')), row=2, col=1)
    fig.add_trace(go.Scatter(x=df['fillTransactDttm'], y=df['fillLimitRefUPrc'], name='Underlier',
                                 line = dict(color='black', width=1, dash='dot')), secondary_y=True, row=2, col=1)

    for t in df.index:
            if df.loc[t, 'orderSide'] == 'Buy':
                fig.add_annotation(dict(
                        x=df.loc[t, 'fillTransactDttm'],
                        y=df.loc[t, 'fillPrice'],
                        ax=0,
                        ay=(50 * df.loc[t, 'fillQuantity'] / max_fill) * scale_arrows + 25 * (1 - scale_arrows),
                        xref='x',
                        yref='y',
                        showarrow=True,
                        arrowcolor='green',
                        arrowsize=2,
                        arrowwidth=1,
                        arrowhead=1
                ))
            else:
                fig.add_annotation(dict(
                        x=df.loc[t, 'fillTransactDttm'],
                        y=df.loc[t, 'fillPrice'],
                        ax=0,
                        ay=-(50 * df.loc[t, 'fillQuantity'] / max_fill) * scale_arrows - 25 * (1 - scale_arrows),
                        xref='x',
                        yref='y',
                        showarrow=True,
                        arrowcolor='red',
                        arrowsize=2,
                        arrowwidth=1,
                        arrowhead=1
                ))
    fig.update_xaxes(range=[start, df.loc[df.index[-1],'fillTransactDttm']])
    if pct_y:
        fig.update_yaxes(title='Vol', tickformat='.2%', secondary_y=False, row=1, col=1)
        fig.update_yaxes(title='Bid/Ask Spread', tickformat='.2%', secondary_y=False, row=2, col=1)
    else:
        fig.update_yaxes(title='Price', tickformat='.2f', secondary_y=False, row=1, col=1)
        fig.update_yaxes(title='Bid/Ask Spread', tickformat='.2f', secondary_y=True, row=2, col=1)
    fig.update_yaxes(title='Cumulative Fills', tickformat=',.0f', showgrid = False, secondary_y=True, row=1, col=1)
    fig.update_yaxes(title='Underlier Price', tickformat=',.0f', showgrid = False, secondary_y=True, row=2, col=1)
    fig.update_layout(height=1000, width=1000)
    fig.show()
    
plot_fill_graph(df_vol, True)